회귀분석에 사용되는 데이터는 그 자체로 사용하기 보다는 스케일링이나 함수 변환 등의 전처리 과정을 거치는 경우가 많다. 전처리 과정은 공분산 행렬의 조건을 향상시키거나 데이터 간의 관계를 선형 모형에 맞게 바꾸기 위해 사용된다.

## 조건수

조건수(conditional number)는 공분산 행렬  𝑋𝑇𝑋 의 가장 큰 고유치와 가장 작은 고유치의 비율을 뜻한다.

$$\text{condition number} = \dfrac{\lambda_{\text{max}}}{\lambda_{\text{min}}}$$

##### 조건수가 크면 역행렬을 계산할 때 오차가 미치는 영향이 커진다. 

여기에서는 다음 연립방정식을 예로 들어 설명을 하겠다.

$$Ax = b$$

행렬  𝐴 가 단위 행렬이면 조건수는 가장 작은 경우로 조건수의 값이 1이다.

$$\text{cond}(I) = 1$$

In [1]:
import numpy as np

In [2]:
A = np.eye(4)

In [3]:
A

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

이 행렬  𝐴 와 곱해져서 상수 벡터  𝑏 가 되는 벡터  𝑥 를 역행렬  𝐴−1 을 사용하여 계산할 수 있다. 이 예에서는 상수 벡터  𝑏 가 1-벡터이다.

In [5]:
b = np.ones(4)
np.linalg.solve(A, b)  

array([1., 1., 1., 1.])

만약 상수 벡터에 약간의 오차가 있었다면 연립방정식의 해에도 동일한 수준의 오차가 발행한다.

In [6]:
np.linalg.solve(A + 0.001 * np.eye(4), b)

array([0.999001, 0.999001, 0.999001, 0.999001])

In [8]:
A + 0.001 * np.eye(4)

array([[1.001, 0.   , 0.   , 0.   ],
       [0.   , 1.001, 0.   , 0.   ],
       [0.   , 0.   , 1.001, 0.   ],
       [0.   , 0.   , 0.   , 1.001]])

In [10]:
0.001 * np.eye(4)

array([[0.001, 0.   , 0.   , 0.   ],
       [0.   , 0.001, 0.   , 0.   ],
       [0.   , 0.   , 0.001, 0.   ],
       [0.   , 0.   , 0.   , 0.001]])

다음과 같은 행렬을 생각하자. 이 행렬은 4차 힐버트 행렬로 조건수가 15000이 넘는다. 이렇게 연립방정식을 이루는 행렬의 조건수가 커지면 상수항 오차가 작은 경우에도 해의 오차가 커지게 된다.

In [15]:
import scipy as sp
import scipy.linalg

In [16]:
A = sp.linalg.hilbert(4) 
A

array([[1.        , 0.5       , 0.33333333, 0.25      ],
       [0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.25      , 0.2       , 0.16666667, 0.14285714]])

In [17]:
np.linalg.cond(A)

15513.738738929038

이 행렬과 곱해져서 상수 벡터가 되는 벡터를 역행렬을 사용하여 찾으면 다음과 같다.

In [18]:
sp.linalg.solve(A, b)

array([  -4.,   60., -180.,  140.])

조건수가 크면 약간의 오차만 있어도 해가 전혀 다른 값을 가진다. 따라서 조건수가 크면 회귀분석을 사용한 예측값도 오차가 커지게 된다.

In [20]:
sp.linalg.solve(A + 0.0001 * np.eye(4), b)

array([ -0.58897672,  21.1225671 , -85.75912499,  78.45650825])

## 회귀분석과 스케일링

회귀분석에서 조건수가 커지는 경우는 크게 두 가지가 있다.

1. 변수들의 단위 차이로 인해 숫자의 스케일이 크게 달라지는 경우. 이 경우에는 스케일링(scaling)으로 해결한다.

2. 다중 공선성 즉, 상관관계가 큰 독립 변수들이 있는 경우, 이 경우에는 변수 선택이나 PCA를 사용한 차원 축소 등으로 해결한다.

보스턴 집값 데이터의 경우 회귀분석을 하면 조건수가 15,000 정도로 크게 나온다.

In [21]:
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()

dfX = pd.DataFrame(boston.data, columns=boston.feature_names)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])
df = pd.concat([dfX, dfy], axis=1)

In [22]:
import statsmodels.api as sm

model1 = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df)
result1 = model1.fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Tue, 13 Aug 2019   Prob (F-statistic):          6.72e-135
Time:                        11:35:17   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     36.4595      5.103      7.144      0.0

조건수(Cond. No.) = 1,5100

그 이유는 각 독립 변수들이 0.1 단위부터 수백 단위까지 제각각의 크기를 가지고 있기 때문이다.  

In [27]:
dfX.describe().iloc[[3, 7], :]  #각 독립변수별로 최대값, 최소값의 차이가 크다

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
min,0.00632,0.0,0.46,0.0,0.385,3.561,2.9,1.1296,1.0,187.0,12.6,0.32,1.73
max,88.97620,100.0,27.74,1.0,0.871,8.780,100.0,12.1265,24.0,711.0,22.0,396.90,37.97


이렇게 조건수가 크면 모수 추정 오차가 증폭될 가능성이 커진다. 이 효과를 확실하게 보기 위하여 일부러 다음과 같이 조건수를 더 크게 만들어 보았다. 이 상태로 선형 회귀분석을 하면 성능이 급격하게 떨어진다.

summary 리포트에서 성능을 나타내는 지표는 R-squared라는 이름의 결정계수다. 결정계수에 대해서는 분산분석에서 자세히 설명한다. 결정계수 값이 원래의 모형에서는 0.741이었는데 조건이 나빠지면서 0.332로 감소한 것을 볼 수 있다.

In [28]:
dfX2 = dfX.copy()
dfX2["TAX"] *= 1e13  # TAX * 100000000000000
df2 = pd.concat([dfX2, dfy], axis=1)

In [30]:
model2 = sm.OLS.from_formula("MEDV ~ " + "+".join(boston.feature_names), data=df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.329
Method:                 Least Squares   F-statistic:                     83.39
Date:                Tue, 13 Aug 2019   Prob (F-statistic):           8.62e-44
Time:                        11:43:41   Log-Likelihood:                -1737.9
No. Observations:                 506   AIC:                             3484.
Df Residuals:                     502   BIC:                             3501.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0038      0.000     -8.543      0.0

StatsModels에서는 모형지정 문자열에서 scale 명령을 사용하여 스케일링을 할 수 있다. 

이 방식으로 스케일을 하면 스케일링에 사용된 평균과 표준편차를 저장하였다가 나중에 predict 명령을 사용할 때도 같은 스케일을 사용하기 때문에 편리하다. 더미 변수인 CHAS는 스케일을 하지 않는다는 점에 주의한다.

In [31]:
feature_names = list(boston.feature_names)
feature_names.remove("CHAS")
feature_names = ["scale({})".format(name) for name in feature_names] + ["CHAS"]
feature_names

['scale(CRIM)',
 'scale(ZN)',
 'scale(INDUS)',
 'scale(NOX)',
 'scale(RM)',
 'scale(AGE)',
 'scale(DIS)',
 'scale(RAD)',
 'scale(TAX)',
 'scale(PTRATIO)',
 'scale(B)',
 'scale(LSTAT)',
 'CHAS']

In [32]:
model3 = sm.OLS.from_formula("MEDV ~ " + "+".join(feature_names), data=df2)
result3 = model3.fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Tue, 13 Aug 2019   Prob (F-statistic):          6.72e-135
Time:                        11:55:30   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         22.3470      0.219    101.

조건수가 10.6으로 떨어지고, R-squared도 0.741로 올라감

In [33]:
model4 = sm.OLS.from_formula("MEDV ~ " + "+".join(feature_names), data=df)
result4 = model4.fit()
print(result4.summary())

                            OLS Regression Results                            
Dep. Variable:                   MEDV   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Tue, 13 Aug 2019   Prob (F-statistic):          6.72e-135
Time:                        11:57:29   Log-Likelihood:                -1498.8
No. Observations:                 506   AIC:                             3026.
Df Residuals:                     492   BIC:                             3085.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         22.3470      0.219    101.

df에 대해 스케일링을 하고 회귀분석을 하면 조건수는 크게 줄어들었으나 R-squared(결정계수)는 바뀌지 않았다.